In [1]:
import torch
from torch import nn
from torch.nn import functional as F

model_name = "google/electra-large-discriminator"

In [2]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

bin c:\Users\Abstract\mambaforge\envs\sentenv2\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


In [3]:
from datasets import load_dataset

imdb = load_dataset("imdb")
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [4]:
from transformers import DataCollatorWithPadding, AutoTokenizer, LlamaTokenizerFast

tokenizer = AutoTokenizer.from_pretrained(model_name)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


def preprocess_function(example):
    tokens = tokenizer(example["text"], truncation=False)
    if len(tokens['input_ids']) > 512:
        tokens['input_ids'] = tokens['input_ids'][:129] + \
            [102] + tokens['input_ids'][-382:]
        tokens['token_type_ids'] = [0]*512
        tokens['attention_mask'] = [1]*512
    return tokens


In [5]:
imdb["test"][12]

{'text': 'I first watched this movie back in the mid/late 80\'s, when I was a kid. We couldn\'t even get all the way through it. The dialog, the acting, everything about it was just beyond lame.<br /><br />Here are a few examples... imagine these spoken real dramatically, way over-acted: "Oreegon? You\'re going to Oreegon? Why would anyone want to go to Oreegon?"<br /><br />"Survivalists? Nobody ever told us about any survivalists!"<br /><br />This movie was SO bad, my sister and I rented it again for her 16th birthday party, just so our friends could sit around and laugh at how awful it was. I don\'t think we were able to finish it then either!',
 'label': 0}

In [6]:
tokenized_imdb = imdb.map(preprocess_function, batched=False)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (532 > 512). Running this sequence through the model will result in indexing errors


In [7]:
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)#.to('cuda')

# model = torch.compile(model)
model

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-23): 24 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (L

In [8]:
training_args = TrainingArguments(
    output_dir="my_awesome_electra_large_spliced",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,
    warmup_steps=100,
    # torch_compile=True,
    # fp16=True,
    # load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

c:\Users\Abstract\mambaforge\envs\sentenv2\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1562 [00:00<?, ?it/s]

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.5028, 'learning_rate': 2e-05, 'epoch': 0.13}
{'loss': 0.2106, 'learning_rate': 1.863201094391245e-05, 'epoch': 0.26}
{'loss': 0.1829, 'learning_rate': 1.72640218878249e-05, 'epoch': 0.38}
{'loss': 0.165, 'learning_rate': 1.5896032831737347e-05, 'epoch': 0.51}
{'loss': 0.1482, 'learning_rate': 1.4528043775649796e-05, 'epoch': 0.64}
{'loss': 0.1394, 'learning_rate': 1.3160054719562245e-05, 'epoch': 0.77}
{'loss': 0.1451, 'learning_rate': 1.1792065663474691e-05, 'epoch': 0.9}


  0%|          | 0/3125 [00:00<?, ?it/s]

{'eval_loss': 0.10218027979135513, 'eval_accuracy': 0.96384, 'eval_runtime': 754.598, 'eval_samples_per_second': 33.13, 'eval_steps_per_second': 4.141, 'epoch': 1.0}
{'loss': 0.1213, 'learning_rate': 1.0424076607387142e-05, 'epoch': 1.02}
{'loss': 0.091, 'learning_rate': 9.05608755129959e-06, 'epoch': 1.15}
{'loss': 0.0988, 'learning_rate': 7.688098495212039e-06, 'epoch': 1.28}
{'loss': 0.0822, 'learning_rate': 6.3201094391244875e-06, 'epoch': 1.41}
{'loss': 0.092, 'learning_rate': 4.952120383036936e-06, 'epoch': 1.54}
{'loss': 0.0894, 'learning_rate': 3.584131326949385e-06, 'epoch': 1.66}
{'loss': 0.1006, 'learning_rate': 2.2161422708618332e-06, 'epoch': 1.79}
{'loss': 0.0767, 'learning_rate': 8.481532147742819e-07, 'epoch': 1.92}


  0%|          | 0/3125 [00:00<?, ?it/s]

{'eval_loss': 0.11105333268642426, 'eval_accuracy': 0.96628, 'eval_runtime': 750.9747, 'eval_samples_per_second': 33.29, 'eval_steps_per_second': 4.161, 'epoch': 2.0}
{'train_runtime': 5724.6705, 'train_samples_per_second': 8.734, 'train_steps_per_second': 0.273, 'train_loss': 0.1472320810017604, 'epoch': 2.0}


TrainOutput(global_step=1562, training_loss=0.1472320810017604, metrics={'train_runtime': 5724.6705, 'train_samples_per_second': 8.734, 'train_steps_per_second': 0.273, 'train_loss': 0.1472320810017604, 'epoch': 2.0})

In [57]:
F.softmax(model(**(tokenizer("This was a fine movie, but I wouldn't watch it again to be honest, a bit too drawn out for my liking. My friends liked it though, so if you like detectives, you may want to watch it, if you have some spare time.", return_tensors="pt").to('cuda'))).logits, dim=1)

tensor([[0.3959, 0.6041]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [59]:
F.softmax(model(**(tokenizer("I enjoyed it.", return_tensors="pt").to('cuda'))).logits, dim=1)

tensor([[0.0083, 0.9917]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [55]:
F.softmax(model(**(tokenizer("I was absolutely extatic when I left the cinema! It was such a relief when I knew that it was over and I won't have to watch this joke of a movie again! Honestly would go to this movie again just to leave as soon as it starts, lol", return_tensors="pt").to('cuda'))).logits, dim=1)

tensor([[0.9930, 0.0070]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [20]:
model.forward(**(tokenizer('hello', return_tensors="pt").to('cuda')))

SequenceClassifierOutput(loss=None, logits=tensor([[-0.2006,  0.1987]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [60]:
torch.cuda.empty_cache()